In [1]:
# !pip install huggingface_hub
#!pip install torcheval

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, MambaForCausalLM
import torch
from torch.utils.data import DataLoader
from transformers import DataCollatorForLanguageModeling
from tqdm import tqdm
from hybrid_model import HybridModel

# from transformers import MambaModel

/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/mamba_ssm/ops/selective_scan_interface.py:163: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/mamba_ssm/ops/selective_scan_interface.py:239: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/mamba_ssm/ops/triton/layer_norm.py:985: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/mamba_ssm/ops/triton/layer_norm.py:1044: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/

In [3]:
# seed, buffer_size = 42, 10_000
# dataset = load_dataset('HuggingFaceFW/fineweb', split='train', streaming=True)
# dataset = dataset.shuffle(seed, buffer_size=buffer_size)

In [4]:
transformer_tokenizer = AutoTokenizer.from_pretrained('EleutherAI/gpt-neo-125M',padding_side="left")
# if transformer_tokenizer.mask_token is None:
#     transformer_tokenizer.add_special_tokens({'mask_token': '[MASK]'})

In [5]:
#dataset = dataset.with_format("torch")
from datasets import load_dataset
dataset = load_dataset("eli5_category", split="train")

In [6]:
if transformer_tokenizer.pad_token is None:
    #transformer_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    transformer_tokenizer.pad_token = transformer_tokenizer.eos_token

In [7]:
# changed the dataset to eli5  It's an English-language dataset of questions and answers gathered from the 
# r/explainlikeimfive subreddit where users ask factual questions requiring paragraph-length or longer answers.


In [8]:
#train-test split
dataset = dataset.train_test_split(test_size=0.2)

In [9]:
train_dataset = dataset["train"].flatten()

In [10]:
max_seq_length =  100
def tokenize(example):
    return transformer_tokenizer(example, truncation=True, max_length=max_seq_length, padding="max_length")

In [11]:
def preprocess_function(examples):
    result = tokenize([" ".join(x) for x in examples["answers.text"]])
    result["labels"] = result["input_ids"].copy()
    return result

In [12]:
tokenized_eli5 = train_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=train_dataset.column_names
    )

Map (num_proc=4):   0%|          | 0/73417 [00:00<?, ? examples/s]

In [13]:
tokenized_eli5

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 73417
})

In [14]:
from transformers import DataCollatorForLanguageModeling
#tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=transformer_tokenizer, mlm=False)

In [15]:
dataloader = DataLoader(tokenized_eli5, batch_size=32, collate_fn=data_collator,shuffle=True)

In [16]:
device = 'cuda:5' if torch.cuda.is_available() else 'cpu'

In [17]:
#device = torch.cuda.device(device=device)

In [18]:
# transformer_model = AutoModelForCausalLM.from_pretrained(
#     'EleutherAI/gpt-neo-125M'
# )
# mamba_model = MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf")
# transformer_model = transformer_model.to(device)
# mamba_model = mamba_model.to(device)

In [19]:
# model.to(device) ? unfreeze : not
hybrid_model = HybridModel(transformer_model=  'EleutherAI/gpt-neo-125M', mamba_model="state-spaces/mamba-130m-hf",proj_type= "gressf", n_hybrid_blocks=2).to(device)
trainer_params = []
# for combiner in model.combiners.parameters():
#     trainer_params.append(combiner)

# for spliter in model.splitters.parameters():
#     trainer_params.append(spliter)
for param in hybrid_model.trans_model.parameters():
    param.requires_grad = False

for param in hybrid_model.mamba_model.parameters():
    param.requires_grad = False


for param in hybrid_model.parameters():
    if param.requires_grad:
       trainer_params.append(param)
    
#trainer_params = torch.T(trainer_params)
print(trainer_params[0])
len(trainer_params)

50257
Parameter containing:
tensor([0.6518], device='cuda:5', requires_grad=True)


21

In [20]:
for param in trainer_params:
    print(param.shape)

torch.Size([1])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([1])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([1])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([1])
torch.Size([768, 768])
torch.Size([768])
torch.Size([768, 768])
torch.Size([768])
torch.Size([50257, 768])


In [21]:
hybrid_model.train()
optimizer = torch.optim.AdamW(params=trainer_params, lr=1e-5)
for e in range(3):
    # shuffle the dataset
    for i, batch in enumerate(dataloader):
        #batch = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}
        outputs = hybrid_model(input_ids = batch['input_ids'].to(device), attention_mask = batch['attention_mask'].to(device), labels = batch['labels'].to(device))
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if i % 10 == 0:
            print(f"loss: {loss}")

loss: 23851.171875
loss: 18653.875
loss: 16561.623046875
loss: 15655.5009765625
loss: 14120.4892578125
loss: 12729.478515625
loss: 11948.5654296875
loss: 10761.2705078125
loss: 10848.765625
loss: 10257.875
loss: 9008.8740234375
loss: 8761.685546875
loss: 8831.513671875
loss: 7778.470703125
loss: 7311.8837890625
loss: 6897.130859375
loss: 6821.9921875
loss: 6430.33642578125
loss: 5784.8095703125
loss: 5385.42724609375
loss: 5128.6982421875
loss: 5533.17724609375
loss: 4628.66650390625
loss: 5079.478515625
loss: 4597.1591796875
loss: 4263.53662109375
loss: 4283.58544921875
loss: 4475.01220703125
loss: 3685.18115234375
loss: 3701.989501953125
loss: 3264.80224609375
loss: 3295.57275390625
loss: 3092.925048828125
loss: 3083.652099609375
loss: 3127.760009765625
loss: 2851.076904296875
loss: 2844.7841796875
loss: 2862.594970703125
loss: 2456.5205078125
loss: 2508.8984375
loss: 2357.974853515625
loss: 2618.77490234375
loss: 2398.140380859375
loss: 2272.4033203125
loss: 2605.94970703125
loss: 2

KeyboardInterrupt: 

In [22]:
test_dataset = dataset["test"].flatten()

In [23]:
test = test_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=test_dataset.column_names
    )

Map (num_proc=4):   0%|          | 0/18355 [00:00<?, ? examples/s]

In [24]:
testloader = DataLoader(test, batch_size=32, collate_fn=data_collator,shuffle=True)

In [25]:
from torcheval.metrics.text import Perplexity

In [22]:
metric=Perplexity().to(device)

In [26]:
hybrid_model.eval()
with torch.no_grad():
    # shuffle the dataset
    for i, batch in enumerate(testloader):
        #batch = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}
        
        label = batch['labels'].to(device)
        outputs = hybrid_model(input_ids = batch['input_ids'].to(device), attention_mask = batch['attention_mask'].to(device), labels = label)
        loss = outputs.loss
        #print(outputs.logits.shape)
        #print(label.shape)
        # Check for NaN or Inf values in the logits
        # if torch.isnan(outputs.logits).any() or torch.isinf(outputs.logits).any():
        #     print("NaN or Inf values found in logits")
        #     continue
        
        # Normalize the logits
        #logit = outputs.logits.reshape(8, 100, -1)
        #logit = logit - logit.max(dim=-1, keepdim=True)[0]  # Subtract max for numerical stability
        #probs = torch.softmax(logit, dim=-1)

        # logit =  outputs.logits.reshape(8,100,-1)
        # probs = torch.softmax(logit, dim=-1)
        #metric.update(probs,label)
        #pp = metric.compute().item()
        if i % 10 == 0:
            print(f"loss: {loss}")
            #print(f"Perplexity:{pp}")

loss: 0.8328383564949036
loss: 0.9889116287231445
loss: 0.5925096273422241
loss: 0.569998025894165
loss: 1.5449453592300415
loss: 0.8924263119697571
loss: 1.0493661165237427
loss: 0.7720130681991577
loss: 1.451814889907837
loss: 1.2217905521392822
loss: 1.1313555240631104
loss: 2.4265832901000977
loss: 0.5735677480697632
loss: 1.2914550304412842
loss: 1.5701082944869995
loss: 1.2303215265274048
loss: 2.3234941959381104
loss: 1.7051411867141724
loss: 0.7536911964416504
loss: 1.0222257375717163
loss: 1.0829304456710815
loss: 1.0872108936309814
loss: 0.8985335230827332
loss: 1.475062608718872
loss: 0.8182273507118225
loss: 0.7591800093650818
loss: 1.0445882081985474
loss: 0.9221876263618469
loss: 0.43152621388435364
loss: 1.504196047782898
loss: 0.5957746505737305
loss: 0.45329245924949646
loss: 0.7991347908973694
loss: 0.649341344833374
loss: 0.8254817128181458
loss: 0.4402911961078644
loss: 1.8027499914169312
loss: 1.0126423835754395
loss: 2.499173641204834
loss: 1.3067868947982788
loss

In [ ]:
#save partial weight ; only the projector
#torch.save(trainer_params, "projector_eli5.pth")

In [27]:
torch.save(hybrid_model.state_dict(), "hybrid_model_eli5.pth")

In [ ]:
# transformer_model = AutoModelForCausalLM.from_pretrained(
#     'EleutherAI/gpt-neo-125M'
# )
# mamba_model = MambaForCausalLM.from_pretrained("state-spaces/mamba-130m-hf")
# transformer_model = transformer_model.to(device)
# mamba_model = mamba_model.to(device)

In [4]:
#hybrid_model = HybridModel(transformer_model='EleutherAI/gpt-neo-125M', mamba_model="state-spaces/mamba-130m-hf",proj_type= "gressf", n_hybrid_blocks=2).to(device)
#state_dict_loaded  = torch.load("./hybrid_model_eli5.pth", weights_only=True)
#hybrid_model.load_state_dict(state_dict_loaded)

50257


In [7]:
# ! pip install ipywidgets

In [31]:
from huggingface_hub import login
login(token='')


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /u/r/i/riyad/.cache/huggingface/token
Login successful


In [32]:
hybrid_model.push_to_hub("RioHassen/manticore-hybrid-gptneo-mamba")

model.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/RioHassen/manticore-hybrid-gptneo-mamba/commit/64ee8bb9b9cf4a1d574d6be18191a8abdd85d8ec', commit_message='Push model using huggingface_hub.', commit_description='', oid='64ee8bb9b9cf4a1d574d6be18191a8abdd85d8ec', pr_url=None, repo_url=RepoUrl('https://huggingface.co/RioHassen/manticore-hybrid-gptneo-mamba', endpoint='https://huggingface.co', repo_type='model', repo_id='RioHassen/manticore-hybrid-gptneo-mamba'), pr_revision=None, pr_num=None)

In [ ]:
hybrid_model.eval()
with torch.no_grad():
    # shuffle the dataset
    for i, batch in enumerate(testloader):
        #batch = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}
        outputs = hybrid_model(input_ids = batch['input_ids'].to(device), attention_mask = batch['attention_mask'].to(device), labels = batch['labels'].to(device))
        loss = outputs.loss
        if i % 10 == 0:
            print(f"loss: {loss}")

AttributeError: 'collections.OrderedDict' object has no attribute 'eval'

In [ ]:
def tokenize_function(examples):
    return transformer_tokenizer(examples["text"],truncation=True)
tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'token_count'])
# Create the data collator for language modeling
data_collator = DataCollatorForLanguageModeling(tokenizer=transformer_tokenizer, mlm=False)

# Create the dataloader
dataloader = DataLoader(tokenized_dataset, batch_size=8, collate_fn=data_collator)

In [ ]:
for token in tokenized_dataset:
    print(token['input_ids'])
    print(token.keys())
    break

tensor([ 1532,   345,   467,  7374,   736, 41291,    11,  6729,  1477,  2577,
          278,    11,   393,  3817,  5718,  1586,    11,   345,   389,  1016,
          284,   765,   284,  2222,   379,  1551,   530,  5166,   286,   285,
        34978,    13,  2773,   661,   481,   772,  2222,   734, 14729,   287,
         1339,   530,  3011,  9583,    13, 31342,    11,   314,  2222,   257,
         5166,   286,   285, 34978,   329, 23125,   290,   257,  5166,   286,
        18051,   329,  2087, 50003,    13,   198,  2215, 17246,   285, 34978,
          290, 18051,   329,  7374,  3403,    11,   994,   389,   257,  3155,
          286,  1276,    12, 14150,  3033,   326,   345,   815,   804,   329,
           25,   198,  8413, 21985,  9493,   364,    25,   921,   761,   257,
          285,  2621,   393, 29144,  1080,   326,   468,  9493,   364,   290,
          257, 35973,  7582,   523,   345,   836,   447,   247,    83,   625,
        25080,   290, 15488,    13, 19372,   265,   287,  7374, 

/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/datasets/formatting/torch_formatter.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(value, **{**default_dtype, **self.torch_tensor_kwargs})


In [ ]:
# from projector import Combiner
# c = Combiner(10,10).to(device)

# c.in_proj1.weight.device

# cm = torch.nn.ModuleList([Combiner(10, 10) for _ in range(12)]).to(device)




In [ ]:

device = 'cuda' if torch.cuda.is_available() else 'cpu' 

for param in transformer_backbone.parameters():
    param.requires_grad = False

for param in mamba_backbone.parameters():
    param.requires_grad = False
    
model = HybridModel(transformer_backbone, mamba_backbone,device) 

# model.to(device) ? unfreeze : not
model.train()

trainer_params = []
for combiner in model.combiners.parameters():
    trainer_params.append(combiner)

for spliter in model.splitters.parameters():
    trainer_params.append(spliter)
optimizer = torch.optim.AdamW(params=model.parameters(), lr=1e-5)
for e in range(3):
    # shuffle the dataset
    dataset.set_epoch(e)
    for i, batch in enumerate(dataloader):
        if i == 5:
            break
        #batch = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}
        outputs = model(input_data = batch['input_ids'])
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if i % 10 == 0:
            print(f"loss: {loss}")

WEIGHT cuda:0


/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/datasets/formatting/torch_formatter.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(value, **{**default_dtype, **self.torch_tensor_kwargs})


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
dataset.__getitem__

<bound method Dataset.__getitem__ of IterableDataset({
    features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'token_count'],
    n_shards: 23781
})>

In [ ]:
#dataloader = DataLoader(tokenized_dataset,batch_size= 32, collate_fn=DataCollatorForLanguageModeling(transformer_tokenizer,mlm=False))




  0%|          | 0/5 [00:00<?, ?it/s]/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/datasets/formatting/torch_formatter.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(value, **{**default_dtype, **self.torch_tensor_kwargs})
  0%|          | 0/5 [00:02<?, ?it/s]

{'language_score': tensor([0.9335, 0.9738, 0.9514, 0.9592, 0.8248, 0.9719, 0.8839, 0.9658, 0.9717,
        0.8748, 0.9487, 0.9108, 0.9771, 0.8554, 0.9519, 0.9758],
       device='cuda:0'), 'token_count': tensor([539, 391, 119, 658,  71, 192, 352, 391, 796, 513, 109, 104, 294, 207,
        194, 629], device='cuda:0')}


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)

In [ ]:
model.splitters[0].out_proj1.weight.device

device(type='cuda', index=0)

In [ ]:
model.transformer_model.device

device(type='cuda', index=0)

In [ ]:
t['input_ids'].to(device)

tensor([[50256, 50256, 50256,  ...,   285,  2621, 19679],
        [50256, 50256, 50256,  ...,  1708,  6050,    30],
        [50256, 50256, 50256,  ...,   286, 11278,    13],
        ...,
        [50256, 50256, 50256,  ...,    13,  1157, 22199],
        [50256, 50256, 50256,  ...,    13,   447,   251],
        [50256, 50256, 50256,  ...,   329,  3555,     0]], device='cuda:0')

In [ ]:

optimizer = torch.optim.AdamW(params=trainer_params , lr=1e-5)
for epoch in range(3):
    dataset.set_epoch(epoch)
    for i, batch in enumerate(tqdm(batched_dataset, total=5)):
        if i == 5:
            break

        #batch = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}
        print(batch)
        t = transformer_tokenizer(batch["text"], padding="max_length", truncation=True, return_tensors= "pt")
        outputs = transformer_backbone(t['input_ids'])
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        if i % 10 == 0:
            print(f"loss: {loss}")

  0%|          | 0/5 [00:00<?, ?it/s]/u/r/i/riyad/miniconda3/lib/python3.12/site-packages/datasets/formatting/torch_formatter.py:78: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(value, **{**default_dtype, **self.torch_tensor_kwargs})
  0%|          | 0/5 [00:02<?, ?it/s]


{'text': ['If you go winter backpacking, snowshoeing, or mountaineering, you are going to want to bring at least one pair of mittens. Some people will even bring two pairs in case one gets wet. Personally, I bring a pair of mittens for warmth and a pair of gloves for added dexterity.\nWhen selecting mittens and gloves for winter conditions, here are a couple of must-have features that you should look for:\nRemovable liners: You need a mitten or glove system that has liners and a removable shell so you don’t overheat and sweat. Sweat in winter is nasty because it freezes in place and chills you, so you have to be extra vigilant about removing layers when you get hot and slow your activity levels to the point where you can stay dry. You can also put removable liners in your sleeping bag at night to dry them.\nWaterproof, breathable shell: When you sweat you need a shell fabric that has the ability to vent moisture and prevent it from getting in. Higher denier (tougher) Gore-Tex shells wo

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)